In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from keras.preprocessing import text
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import datetime
from sklearn import metrics
!pip install tensorflow_text==2.4.3
import tensorflow_text as text

In [ ]:
import pandas as pd
import numpy as np
import re

# plotting
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# read data
df = pd.read_csv('../input/mbti-type/mbti_1.csv') 

In [ ]:
def get_types(row):
    t=row['type']

    I = 0; N = 0
    T = 0; J = 0
    
    if t[0] == 'I': I = 1
    elif t[0] == 'E': I = 0
    else: print('I-E incorrect')
        
    if t[1] == 'N': N = 1
    elif t[1] == 'S': N = 0
    else: print('N-S incorrect')
        
    if t[2] == 'T': T = 1
    elif t[2] == 'F': T = 0
    else: print('T-F incorrect')
        
    if t[3] == 'J': J = 1
    elif t[3] == 'P': J = 0
    else: print('J-P incorrect')
    return pd.Series( {'IE':I, 'NS':N , 'TF': T, 'JP': J }) 

df = df.join(df.apply (lambda row: get_types (row),axis=1))
df.head(5)

In [ ]:
#转换成00001111
b_Pers = {'I':0, 'E':1, 'N':0, 'S':1, 'F':0, 'T':1, 'J':0, 'P':1}
b_Pers_list = [{0:'I', 1:'E'}, {0:'N', 1:'S'}, {0:'F', 1:'T'}, {0:'J', 1:'P'}]

def translate_personality(personality):
    # transform mbti to binary vector
    
    return [b_Pers[l] for l in personality]

def translate_back(personality):
    # transform binary vector to mbti personality
    
    s = ""
    for i, l in enumerate(personality):
        s += b_Pers_list[i][l]
    return s

# Check ...
d = df.head(4)
list_personality_bin = np.array([translate_personality(p) for p in d.type])
print("Binarize MBTI list: \n%s" % list_personality_bin)


In [ ]:
##### Compute list of subject with Type | list of comments 
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk import word_tokenize

# We want to remove these from the psosts
unique_type_list = ['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
       'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ']
  



# Lemmatize
stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()

# Cache the stop words for speed 
cachedStopWords = stopwords.words("english")

def pre_process_data1(data, remove_stop_words=True, remove_mbti_profiles=True):

    list_personality = []
    list_posts = []
    len_data = len(data)
    i=0
    
    for row in data.iterrows():
        i+=1
        if (i % 500 == 0 or i == 1 or i == len_data):
            print("%s of %s rows" % (i, len_data))

        ##### Remove and clean comments
        posts = row[1].posts
        temp = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', posts)
        temp = re.sub("[^a-zA-Z]", " ", temp)
        temp = re.sub(' +', ' ', temp).lower()
        if remove_stop_words:
            temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ') if w not in cachedStopWords])
        else:
            temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ')])
            
        if remove_mbti_profiles:
            for t in unique_type_list:
                temp = temp.replace(t,"")

        type_labelized = translate_personality(row[1].type)
        list_personality.append(type_labelized)
        list_posts.append(temp)

    list_posts = np.array(list_posts)
    list_personality = np.array(list_personality)
    return list_posts, list_personality

In [ ]:
list_posts2, list_personality2  = pre_process_data1(df, remove_stop_words=True)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE

# Posts to a matrix of token counts
cntizer = CountVectorizer(analyzer="word", 
                             max_features=1500, 
                             tokenizer=None,    
                             preprocessor=None, 
                             stop_words=None,  
                             max_df=0.7,
                             min_df=0.1) 
#max_df :ignore terms that have a document frequency strictly higher 

# Learn the vocabulary dictionary and return term-document matrix
print("CountVectorizer...")
X_cnt = cntizer.fit_transform(list_posts2)

# Transform the count matrix to a normalized tf or tf-idf representation
tfizer = TfidfTransformer()

print("Tf-idf...")
# Learn the idf vector (fit) and transform a count matrix to a tf-idf representation
X_tfidf =  tfizer.fit_transform(X_cnt).toarray()

In [ ]:


type_indicators = [ "IE: Introversion (I) / Extroversion (E)", "NS: Intuition (N) – Sensing (S)", 
                   "FT: Feeling (F) - Thinking (T)", "JP: Judging (J) – Perceiving (P)"  ]

for l in range(len(type_indicators)):
    print(type_indicators[l])



In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Posts in tf-idf representation
X = X_tfidf
param = {}
param['n_estimators'] = 1000
param['max_depth'] = 2
param['learning_rate'] = 0.2


# Let's train type indicator individually

starttime = datetime.datetime.now()
#long running

for l in range(len(type_indicators)):
    print("%s ..." % (type_indicators[l]))
    
    
    # Let's train type indicator individually
    Y = list_personality2[:,l]

    # split data into train and test sets
    seed = 7
    test_size = 0.33
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    # fit model on training data
    model = XGBClassifier(eval_metric="auc",use_label_encoder=False,**param)
    model.fit(X_train, y_train)

    # make predictions for test data
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    roc = metrics.roc_auc_score(y_test, predictions)
    print("* %s Accuracy: %.2f%%" % (type_indicators[l], accuracy * 100.0))
    print("* %s roc: %.2f%%" % (type_indicators[l], roc * 100.0))
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
import lightgbm as lgb
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
import joblib 
model_lgbm = lgb.LGBMClassifier(objective='binary', metric = 'auc',

learning_rate=0.2, n_estimators=1000, max_depth=2, is_unbalance=True,verbosity=-1)

In [ ]:
import lightgbm as lgb
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics

starttime = datetime.datetime.now()
lgb_predict_result = []
lgb_predict_test = []
type_ORDER = ["IE","NS","FT","JP"]
for l in range(len(type_indicators)):
    print("%s ..." % (type_indicators[l]))
    
    
    # Let's train type indicator individually
    Y = list_personality2[:,l]
    modeltype = "TF-IDF"
    modelOrder = modeltype+type_ORDER[l]+".pkl"
    print(modelOrder)
    

    # split data into train and test sets
    seed = 7
    test_size = 0.33
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
    X_train2, X_test2, y_train2, y_test2 = train_test_split(X, Y, test_size=test_size, random_state=42)
    #lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
    #lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train,free_raw_data=False)
    
    
    # fit model on training data
    evals_result = {}
    
    #LGBModel = lgb.train(LGBMparams2, lgb_train, 50000, 
    #                      valid_sets=[lgb_train, lgb_eval], 
    #                      early_stopping_rounds=100, 
    #                      verbose_eval=500, 
    #                      evals_result=evals_result)
    
    LGBModel = model_lgbm
    LGBModel.fit(X_train, y_train,eval_set=(X_test2, y_test2),eval_metric = 'auc',early_stopping_rounds = 100, verbose = 1000)
    
    joblib.dump(LGBModel, modelOrder)

    # make predictions for test data
    
    starttime2 = datetime.datetime.now()
    y_pred = LGBModel.predict(X_test, num_iteration=LGBModel.best_iteration_)
    endtime2 = datetime.datetime.now()
    
    print (endtime2 - starttime2)
    
    lgb_predict_result.append(y_pred)
    lgb_predict_test.append(y_test)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    roc = metrics.roc_auc_score(y_test, predictions)
    print("* %s Accuracy: %.2f%%" % (type_indicators[l], accuracy * 100.0))
    print("* %s roc: %.2f%%" % (type_indicators[l], roc * 100.0))
    
endtime = datetime.datetime.now()
print (endtime - starttime)

* IE: Introversion (I) / Extroversion (E) Accuracy: 86.45%
* IE: Introversion (I) / Extroversion (E) roc: 75.83%

* NS: Intuition (N) – Sensing (S) Accuracy: 89.77%
* NS: Intuition (N) – Sensing (S) roc: 69.13%

* FT: Feeling (F) - Thinking (T) Accuracy: 84.88%
* FT: Feeling (F) - Thinking (T) roc: 84.79%

* JP: Judging (J) – Perceiving (P) Accuracy: 80.79%
* JP: Judging (J) – Perceiving (P) roc: 78.98%

* 0:01:21.022334

In [ ]:
from catboost import CatBoostClassifier
import lightgbm as lgb
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
X = X_tfidf

cat_params = {
                'eval_metric':'AUC',
                'task_type':'GPU',
                'random_seed':42,
                'learning_rate':0.2,
                'depth':2
                #'colsample_bylevel':0.7,
                } 
predict_result = []
predict_test = []
starttime = datetime.datetime.now()

for l in range(len(type_indicators)):
    print("%s ..." % (type_indicators[l]))
    
    
    # Let's train type indicator individually
    Y = list_personality2[:,l]
    
    
    

    # split data into train and test sets
    seed = 7
    test_size = 0.33
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
    
    # fit model on training data
    cb_model = CatBoostClassifier(**cat_params)
    cb_model.fit(X_train, y_train,
             eval_set=(X_test, y_test),
             use_best_model=True,
             verbose=200,
                plot=True)
    

    # make predictions for test data
    y_pred = cb_model.predict(X_test)
    predict_result.append(y_pred)
    predict_test.append(y_test)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    roc = metrics.roc_auc_score(y_test, predictions)
    print("* %s Accuracy: %.2f%%" % (type_indicators[l], accuracy * 100.0))
    print("* %s roc: %.2f%%" % (type_indicators[l], roc * 100.0))
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
!pip install tensorflow_text==2.4.3
import tensorflow_text as text

In [ ]:
def creatmodel():

    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
    preprocessor = hub.KerasLayer(
        "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
    encoder_inputs = preprocessor(text_input)
    encoder = hub.KerasLayer(
        "https://tfhub.dev/tensorflow/bert_en_wwm_uncased_L-24_H-1024_A-16/4",
        trainable=True)
    outputs = encoder(encoder_inputs)
    pooled_output = outputs["pooled_output"]      # [batch_size, 1024].
    sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 1024].
    embedding_model = tf.keras.Model(text_input, pooled_output)
    return embedding_model
model = creatmodel()

In [ ]:
##### Compute list of subject with Type | list of comments 
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk import word_tokenize

# We want to remove these from the psosts
unique_type_list = ['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
       'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ']
unique_type_list = [x.lower() for x in unique_type_list]




# Lemmatize
stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()

# Cache the stop words for speed 
cachedStopWords = stopwords.words("english")

def pre_process_data(data, remove_stop_words=True, remove_mbti_profiles=True):

    list_personality = []
    list_posts = []
    len_data = len(data)
    i=0
    
    for row in data.iterrows():
        i+=1
        if (i % 1000 == 0 or i == 1 or i == len_data):
            print("%s of %s rows" % (i, len_data))

        ##### Remove and clean comments
        posts = row[1].posts        
        if remove_mbti_profiles:
            for t in unique_type_list:
                temp = posts.replace(t,"")
                
        temp = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', posts)
        temp = re.sub("[^a-zA-Z]", " ", temp)
        temp = re.sub(' +', ' ', temp).lower()
        if remove_stop_words:
            temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ') if w not in cachedStopWords])
        else:
            temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ')])
        
        sentences = tf.constant([temp])
        temp = model(sentences)

        type_labelized = translate_personality(row[1].type)
        list_personality.append(type_labelized)
        list_posts.append(temp)

    list_posts = np.array(list_posts)
    list_personality = np.array(list_personality)
    return list_posts, list_personality

In [ ]:
list_posts, list_personality  = pre_process_data(df, remove_stop_words=True)

In [ ]:
list_posts = list_posts.reshape(8675,1024)

In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Posts in tf-idf representation
X = list_posts
param = {}
param['n_estimators'] = 1000
param['max_depth'] = 2
param['learning_rate'] = 0.2

starttime = datetime.datetime.now()
# Let's train type indicator individually
for l in range(len(type_indicators)):
    print("%s ..." % (type_indicators[l]))
    
    
    # Let's train type indicator individually
    Y = list_personality[:,l]

    # split data into train and test sets
    seed = 7
    test_size = 0.33
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    # fit model on training data
    model = XGBClassifier(eval_metric="auc",use_label_encoder=False,**param)
    model.fit(X_train, y_train)

    # make predictions for test data
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    roc = metrics.roc_auc_score(y_test, predictions)
    print("* %s Accuracy: %.2f%%" % (type_indicators[l], accuracy * 100.0))
    print("* %s roc: %.2f%%" % (type_indicators[l], roc * 100.0))
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
import lightgbm as lgb
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = list_posts


    
lgb_predict_result_bert = []
lgb_predict_test_bert = []
type_ORDER = ["IE","NS","FT","JP"]
starttime = datetime.datetime.now()
for l in range(len(type_indicators)):
    print("%s ..." % (type_indicators[l]))
    
    
    # Let's train type indicator individually
    Y = list_personality[:,l]
    modeltype = "BERT"
    modelOrder = modeltype+type_ORDER[l]+".pkl"
    print(modelOrder)
    
    

    # split data into train and test sets
    seed = 7
    test_size = 0.33
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
    X_train2, X_test2, y_train2, y_test2 = train_test_split(X, Y, test_size=test_size, random_state=42)
    #lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
    #lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train,free_raw_data=False)
    
    
    # fit model on training data
    evals_result = {}
   # LGBModel = lgb.train(LGBMparams2, lgb_train, 50000, 
    #                      valid_sets=[lgb_train, lgb_eval], 
     #                     early_stopping_rounds=100, 
      #                    verbose_eval=500, 
       #                   evals_result=evals_result)
    LGBModel = model_lgbm
    LGBModel.fit(X_train, y_train,eval_set=(X_test2, y_test2),eval_metric = 'auc',early_stopping_rounds = 100, verbose = 250)
    
    
    joblib.dump(LGBModel, modelOrder)
    # make predictions for test data
    
    starttime2 = datetime.datetime.now()
    y_pred = LGBModel.predict(X_test, num_iteration=LGBModel.best_iteration_)
    endtime2 = datetime.datetime.now()
    
    print (endtime2 - starttime2)
    
    lgb_predict_result_bert.append(y_pred)
    lgb_predict_test_bert.append(y_test)
    predictions = [round(value) for value in y_pred]
    
    
    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    roc = metrics.roc_auc_score(y_test, predictions)
    print("* %s Accuracy: %.2f%%" % (type_indicators[l], accuracy * 100.0))
    print("* %s roc: %.2f%%" % (type_indicators[l], roc * 100.0))

endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
from catboost import CatBoostClassifier
import lightgbm as lgb
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
X = list_posts

cat_params = {
                'eval_metric':'AUC',
                'task_type':'GPU',
                'random_seed':42,
                'learning_rate':0.2,
                'depth':2
                #'colsample_bylevel':0.7,
                } 
starttime = datetime.datetime.now()
predict_result_bert = []
predict_test_bert = []
for l in range(len(type_indicators)):
    print("%s ..." % (type_indicators[l]))
    
    
    # Let's train type indicator individually
    Y = list_personality[:,l]
    
    
    

    # split data into train and test sets
    seed = 7
    test_size = 0.33
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
    
    # fit model on training data
    cb_model = CatBoostClassifier(**cat_params)
    cb_model.fit(X_train, y_train,
             eval_set=(X_test, y_test),
             use_best_model=True,
             verbose=200,
                plot=True)
    

    # make predictions for test data
    y_pred = cb_model.predict(X_test)
    predict_result_bert.append(y_pred)
    predict_test_bert.append(y_test)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    roc = metrics.roc_auc_score(y_test, predictions)
    print("* %s Accuracy: %.2f%%" % (type_indicators[l], accuracy * 100.0))
    print("* %s roc: %.2f%%" % (type_indicators[l], roc * 100.0))
    
endtime = datetime.datetime.now()    
print (endtime - starttime)

In [ ]:
#no bert LGBM
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

def getconfusion_matrix(dimension):
    Y = list_personality[:,dimension]
    y_pred = predict_result_bert[dimension]
    y_test = predict_test_bert[dimension]
    
    
    
   
    C2= confusion_matrix(y_test, y_pred)
    plt.matshow(C2, cmap=plt.cm.Blues, alpha=0.5)
    plt.show
    sns.heatmap(C2,annot=True,fmt="d")

In [ ]:
getconfusion_matrix(1)

import lightgbm as lgb
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics

starttime = datetime.datetime.now()
lgb_predict_result = []
lgb_predict_test = []
type_ORDER = ["TF-IDFIE.pkl","TF-IDFNS.pkl","TF-IDFFT.pkl","TF-IDFJP.pkl"]
for l in range(len(type_indicators)):
    print("%s ..." % (type_indicators[l]))
    
    
    # Let's train type indicator individually
    Y = list_personality2[:,l]
    modeltype = "TF-IDF"
    modelOrder = modeltype+type_ORDER[l]+".pkl"
    print(modelOrder)
    

    # split data into train and test sets
    seed = 7
    test_size = 0.33
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
    X_train2, X_test2, y_train2, y_test2 = train_test_split(X, Y, test_size=test_size, random_state=42)
    #lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
    #lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train,free_raw_data=False)
    

    
    LGBModel = joblib.load(type_ORDER[l])

    # make predictions for test data
    
    starttime2 = datetime.datetime.now()
    y_pred = LGBModel.predict(X_test, num_iteration=LGBModel.best_iteration_)
    endtime2 = datetime.datetime.now()
    
    print (endtime2 - starttime2)
    
    lgb_predict_result.append(y_pred)
    lgb_predict_test.append(y_test)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    roc = metrics.roc_auc_score(y_test, predictions)
    print("* %s Accuracy: %.2f%%" % (type_indicators[l], accuracy * 100.0))
    print("* %s roc: %.2f%%" % (type_indicators[l], roc * 100.0))
    
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
import lightgbm as lgb
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
import joblib 
import datetime
X = X_tfidf
starttime = datetime.datetime.now()
lgb_predict_result = []
lgb_predict_test = []
type_ORDER = ["TF-IDFIE.pkl","TF-IDFNS.pkl","TF-IDFFT.pkl","TF-IDFJP.pkl"]
for l in range(len(type_indicators)):
    print("%s ..." % (type_indicators[l]))
    
    
    # Let's train type indicator individually
    Y = list_personality2[:,l]
    modeltype = "TF-IDF"
    modelOrder = modeltype+type_ORDER[l]+".pkl"
    print(modelOrder)
    

    # split data into train and test sets
    seed = 7
    test_size = 0.33
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
    X_train2, X_test2, y_train2, y_test2 = train_test_split(X, Y, test_size=test_size, random_state=42)
    #lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
    #lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train,free_raw_data=False)
    

    
    LGBModel = joblib.load("/kaggle/input/fyp-worklab2/"+type_ORDER[l])

    # make predictions for test data
    
    starttime2 = datetime.datetime.now()
    y_pred = LGBModel.predict(X_test, num_iteration=LGBModel.best_iteration_)
    endtime2 = datetime.datetime.now()
    
    print (endtime2 - starttime2)
    
    lgb_predict_result.append(y_pred)
    lgb_predict_test.append(y_test)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    roc = metrics.roc_auc_score(y_test, predictions)
    print("* %s Accuracy: %.2f%%" % (type_indicators[l], accuracy * 100.0))
    print("* %s roc: %.2f%%" % (type_indicators[l], roc * 100.0))
    
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
#no bert LGBM
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

def lgb_getconfusion_matrix(dimension):
    Y = list_personality2[:,dimension]
    y_pred = lgb_predict_result[dimension]
    y_test = lgb_predict_test[dimension]
    
    
    
   
    C2= confusion_matrix(y_test, y_pred)
    plt.matshow(C2, cmap=plt.cm.Blues, alpha=0.5)
    plt.show
    sns.heatmap(C2,annot=True,fmt="d")
    

In [ ]:
lgb_getconfusion_matrix(0)

In [ ]:
lgb_getconfusion_matrix(1)

In [ ]:
lgb_getconfusion_matrix(2)

In [ ]:
lgb_getconfusion_matrix(3)

In [ ]:
import lightgbm as lgb
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
import joblib 
import datetime
X = list_posts
starttime = datetime.datetime.now()
lgb_predict_result_bert = []
lgb_predict_test_bert = []
type_ORDER = ["BERTIE.pkl","BERTNS.pkl","BERTFT.pkl","BERTJP.pkl"]
for l in range(len(type_indicators)):
    print("%s ..." % (type_indicators[l]))
    
    
    # Let's train type indicator individually
    Y = list_personality2[:,l]
    modeltype = "BERT"
    modelOrder = modeltype+type_ORDER[l]+".pkl"
    print(modelOrder)
    

    # split data into train and test sets
    seed = 7
    test_size = 0.33
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
    X_train2, X_test2, y_train2, y_test2 = train_test_split(X, Y, test_size=test_size, random_state=42)
    #lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
    #lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train,free_raw_data=False)
    

    
    LGBModel = joblib.load("/kaggle/input/fyp-worklab2/"+type_ORDER[l])

    # make predictions for test data
    
    starttime2 = datetime.datetime.now()
    y_pred = LGBModel.predict(X_test, num_iteration=LGBModel.best_iteration_)
    endtime2 = datetime.datetime.now()
    
    print (endtime2 - starttime2)
    
    lgb_predict_result_bert.append(y_pred)
    lgb_predict_test_bert.append(y_test)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    roc = metrics.roc_auc_score(y_test, predictions)
    print("* %s Accuracy: %.2f%%" % (type_indicators[l], accuracy * 100.0))
    print("* %s roc: %.2f%%" % (type_indicators[l], roc * 100.0))
    
endtime = datetime.datetime.now()
print (endtime - starttime)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
def lgb_getconfusion_matrix_bert(dimension):
    Y = list_personality[:,dimension]
    y_pred = lgb_predict_result_bert[dimension]
    y_test = lgb_predict_test_bert[dimension]
   
    
    
    
   
    C2= confusion_matrix(y_test, y_pred)
    plt.matshow(C2, cmap=plt.cm.Blues, alpha=0.5)
    plt.show
    sns.heatmap(C2,annot=True,fmt="d")

In [ ]:
lgb_getconfusion_matrix_bert(0)

In [ ]:
lgb_getconfusion_matrix_bert(1)

In [ ]:
lgb_getconfusion_matrix_bert(2)

In [ ]:
lgb_getconfusion_matrix_bert(3)